# Gene Expression Studies of Response to Influenza Virus Infection Using Two Bulk RNA-Seq Datasets

## Overview

This tutorial demonstrates how to run an RNA-Seq workflow using a *Mus musculus* dataset. Steps in the workflow include read trimming, quality control, read mapping, and counting mapped reads per gene to quantitate gene expression.

All outputs used in [Tutorial 2](https://github.com/King-Laboratory/scRNASeq-miRNASeq-and-TF-Network-Analysis/blob/bda75860ace82cf180a6f9eae115ebaf2eabc5f9/Bulk_RNA-Seq_Tutorials/Bulk_RNA-Seq_Mouse/Tutorial_2_DEG_mouse.ipynb) for DEG analysis were created using this extended full dataset tutorial workflow.

![Mouse Bulk RNA-seq workflow](images/Mouse_workflow.png)

## Learning Objectives

- Explore an example Bulk RNA-sequencing dataset
- Understand the workflow of generating read counts, including:
    - Accessing SRA metadata
    - Quality control
    - Adapter trimming
    - Read mapping
    - Counting mapped reads
    - Quanitify gene expression levels
- Report expression of the top 10 highly expressed genes
- Combine read count files and store in AWS S3 bucket

## STEP 1: Getting Started

<div class="alert alert-block alert-warning"> NOTE: This Jupyter Notebook was developed to run on GCP.</div>

### Without Container: Install Miniforge and Workflow Tools

Miniforge is a lightweight Conda distribution that offers a streamlined installation process and efficient package management. It provides access to a vast repository of packages.

The following code performs these steps:
- Downloads Miniforge or Mambaforge (you can use either based on preference)
- Installs Miniforge (or Mambaforge) - no need to install conda since mamba will be available immediately
- Using miniforge and bioconda, installs the tools that will be used in this tutorial

<div class="alert alert-block alert-info">Tip: If using the Miniforge install, run the following code cells by removing the %%script false command. </div>

In [1]:
# Download Miniforge or Mambaforge (you can use either based on preference)
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh

# Install Miniforge (or Mambaforge) - no need to install conda since mamba will be available immediately
!bash Miniforge3-$(uname)-$(uname -m).sh -b -u -p $HOME/miniforge > /dev/null
!date +"%T"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 81.7M  100 81.7M    0     0   164M      0 --:--:-- --:--:-- --:--:--  164M
14:55:43


Next, using mambaforge and bioconda, install the tools that will be used in this tutorial.

In [2]:
# Update PATH to point to the Miniforge (or Mambaforge) bin files
import os
os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/miniforge/bin"

#now we can easily use 'mamba' command to install software 
!mamba install -y -c conda-forge -c bioconda trimmomatic fastqc multiqc sql-magic entrez-direct gffread parallel-fastq-dump sra-tools sql-magic pyathena samtools star rsem entrez-direct subread pigz -y > /dev/null

warning  libmamba [click-8.3.1-pyh8f84b5b_1] The following files were already present in the environment:
    - lib/python3.10/site-packages/click/__init__.py
    - lib/python3.10/site-packages/click/_compat.py
    - lib/python3.10/site-packages/click/_termui_impl.py
    - lib/python3.10/site-packages/click/_textwrap.py
    - lib/python3.10/site-packages/click/_winconsole.py
    - lib/python3.10/site-packages/click/core.py
    - lib/python3.10/site-packages/click/decorators.py
    - lib/python3.10/site-packages/click/exceptions.py
    - lib/python3.10/site-packages/click/formatting.py
    - lib/python3.10/site-packages/click/globals.py
    - lib/python3.10/site-packages/click/parser.py
    - lib/python3.10/site-packages/click/py.typed
    - lib/python3.10/site-packages/click/shell_completion.py
    - lib/python3.10/site-packages/click/termui.py
    - lib/python3.10/site-packages/click/testing.py
    - lib/python3.10/site-packages/click/types.py
    - lib/python3.10/site-packages/click/

---------------------------------------
## If running from a container, as noted above, start with <b> STEP 2 </b> below:
## STEP 2: Define Threads & Setup Directory Structures

Specify the number of available threads based on the VM. This is useful for later tools such as trimmomatic, or STAR.

In [3]:
import multiprocessing
import os

num_cores = multiprocessing.cpu_count()
THREADS = max(1, num_cores - 1)

print("Number of threads:", THREADS)
os.environ["THREADS"] = str(THREADS)

Number of threads: 31


Create a set of directories in the sra-data-athena to store the reads, reference sequence files, and output files.

In [4]:
!cd $HOMEDIR
!echo $PWD
!mkdir -p data
!mkdir -p data/raw_fastq
!mkdir -p data/trimmed
!mkdir -p data/fastqc
!mkdir -p data/fastqc_trimmed
!mkdir -p data/reference
!mkdir -p data/aligned_bam
!mkdir -p data/rsem_reference/mouse_rsem_reference
!mkdir -p data/rsem_output
!mkdir -p data/reference/STAR_index

/home/jupyter/Bulk-and-Single-Cell-RNAseq/Bulk_RNA-Seq_Tutorials/Bulk_RNA-Seq_Mouse


## STEP 3: Downloading relevant FASTQ files



Next we will need to download the relevant fastq files.

Because these files can be large, the process of downloading the fastq files can be quite lengthy.

We will be downloading the fastq files using the SRA Toolkit from the NCBI's SRA (Sequence Read Archive). However, first we need to define the associated accession numbers in order to download.

### STEP 3.1: Finding run accession numbers.


The SRA stores sequence data in terms of runs, (SRR stands for Sequence Read Run). For the studies we are interested in, there is a single run ID for each sample. 

The mouse strain study has 36 samples and is represented in the Gene Expression Omnibus (GEO) and BioProject databases:
    
   - GEO Dataset ID: GSE66040
    
   - BioProject ID: PRJNA275751
    
The mouse lung resident cell type study has 43 samples and is represented in the Gene Expression Omnibus (GEO) and BioProject databases:
    
   - GEO Dataset ID: GSE165299
    
   - BioProject ID: PRJNA694034

Since you are working in two different teams, you will analyze a subset of 8 samples each. There are files containing SRR IDs for groups of 8 samples from these two experiments. You will use one of these files for your subsequent analyses.

`accs_GSE66040_set1.txt`
                                                                                                                             
`accs_GSE66040_set2.txt`
                                                                                                                             
`accs_GSE66040_set3.txt`
                                                                                                                             
`accs_GSE66040_set4.txt`
                                                                                                                             
`accs_GSE66040_set5.txt`

                                                                                                                             
`accs_GSE165299_set1.txt`
                                                                                                                             
`accs_GSE165299_set2.txt`
                                                                                                                             
`accs_GSE165299_set3.txt`
                                                                                                                             
`accs_GSE165299_set4.txt`
                                                                                                                             
`accs_GSE165299_set5.txt`
                                                                                                                             
`accs_GSE165299_set6.txt`
                                                                                                                             
You will substitute your file above (e.g., "accs_GSE66040_set1.txt" for "accs.txt") in the code blocks that follow.

### STEP 3.2 Downloading multiple files using the SRA-toolkit.

The code uses prefetch to download multiple SRA files in parallel. It reads the list of SRR IDs from accs.txt, uses xargs to execute prefetch for each ID, and specifies the output directory and the -f option to create FASTQ files in the same directory as the SRA files. To speed up the download the code uses -P $THREADS option allowing parallel execution using the specified number of threads.

In [ ]:
!cat accs.txt | xargs -P $THREADS -I {} prefetch {} -O data/raw_fastq -f yes

### STEP 3.3 Converting Multiple SRA files to Fastq


In this step, the SRA files will be processed in parallel using parallel-fastq-dump. Each SRR ID from accs.txt will be read, and xargs will be used to execute parallel-fastq-dump for each SRA ID. This will result in the creation of two paired-end FASTQ files for each SRR ID, which will be compressed into a .gz file to save space.

In [ ]:
#process with parallel-fastq-dump using piping
!cat accs.txt | xargs -I {} parallel-fastq-dump -O data/raw_fastq/ --tmpdir . --threads $THREADS --gzip --split-files --sra-id {}

As before, it is good practice to turn .fastq files into .fastq.gz files to save space.

In our case, we will actually need to concatenate the fastq files later on, and so will zip after this.

The non-redundant SRA files can also be deleted to save more space.

In [ ]:
#find and delete all SRR subfolders in the raw_fastq directory
!find data/raw_fastq -type d -name 'SRR*' -exec rm -rf {} \;

### STEP 3.6 Download reference transcriptome files that will be used by STAR


This step downloads and prepares the reference data needed for your RNA-Seq analysis. It retrieves three essential files:

- **Mouse genome (Mus_musculus.GRCm39.dna.primary_assembly.fa.gz)**: This compressed FASTA file contains the complete mouse genome sequence, that will be used as the reference for aligning your RNA-seq reads.
- **Mouse gene annotations (Mus_musculus.GRCm39.104.gtf.gz)**: This compressed GTF file provides information about the genes and transcripts in the mouse genome, including their locations and structures. This data will crucial for interpreting the aligned RNA-Seq reads and understanding what genes are expressed in each.
- **Mouse feature table (GCF_000001635.27_GRCm39_feature_table.txt.gz)**: This compressed table provides additional annotations for the mouse genome features, potentially including information about gene functions and pathways. This step will further used to analyze the differential gene expression (DEG) analysis. 

In [6]:
!wget ftp://ftp.ensembl.org/pub/release-104/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna.primary_assembly.fa.gz -O data/reference/mouse_genome.fa.gz
!wget ftp://ftp.ensembl.org/pub/release-104/gtf/mus_musculus/Mus_musculus.GRCm39.104.gtf.gz -O data/reference/mouse_annotation.gtf.gz
!wget -O data/reference/mouse_feature_table.txt.gz https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/635/GCF_000001635.27_GRCm39/GCF_000001635.27_GRCm39_feature_table.txt.gz

--2026-01-07 15:26:12--  ftp://ftp.ensembl.org/pub/release-104/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna.primary_assembly.fa.gz
           => ‘data/reference/mouse_genome.fa.gz’
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.169|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/release-104/fasta/mus_musculus/dna ... done.
==> SIZE Mus_musculus.GRCm39.dna.primary_assembly.fa.gz ... 806418890
==> PASV ... done.    ==> RETR Mus_musculus.GRCm39.dna.primary_assembly.fa.gz ... done.
Length: 806418890 (769M) (unauthoritative)

Mus_musculus.GRCm39 100%[===================>] 769.06M  39.0MB/s    in 21s     

2026-01-07 15:26:34 (37.3 MB/s) - ‘data/reference/mouse_genome.fa.gz’ saved [806418890]

--2026-01-07 15:26:34--  ftp://ftp.ensembl.org/pub/release-104/gtf/mus_musculus/Mus_musculus.GRCm39.104.gtf.gz
           => ‘data/referenc

In [7]:
!gunzip -f data/reference/mouse_genome.fa.gz 
!gunzip -f data/reference/mouse_annotation.gtf.gz
!gunzip -f data/reference/mouse_feature_table.txt.gz

### STEP 3.4: Copy data file for Trimmomatic

One of the functions of Trimmomatic is to trim adapter sequences unique to each sequencing platform. These adapter sequences are typically located within the trimmomatic installation directory in a folder called adapters.

Directories of packages within conda installations can be confusing, so in the case of using conda with Trimmomatic, it may be easier to simply download or create a file with the relevant adapter sequences and store it in an easy to find directory.

In [8]:
!cp TruSeq3-SE.fa data/trimmed/.

/bin/bash: line 1: aws: command not found


### STEP 4: Run FastQC

FastQC is an invaluable tool that allows you to evaluate whether there are problems with a set of reads. For example, it will provide a report of whether there is any bias in the sequence composition of the reads.

The below code may take a while to run. To make it run faster we can use threads to speed up the process.

In [9]:
# Run fastqc for forward reads in parallel
!cat accs.txt | xargs -P $THREADS -I {} fastqc "data/raw_fastq/{}_1.fastq.gz" -o data/fastqc/


Skipping 'data/raw_fastq/SRR1810171.fastq.gz' which didn't exist, or couldn't be read
Skipping 'data/raw_fastq/SRR1810173.fastq.gz' which didn't exist, or couldn't be read
Skipping 'data/raw_fastq/SRR1810174.fastq.gz' which didn't exist, or couldn't be read
Skipping 'data/raw_fastq/SRR1810172.fastq.gz' which didn't exist, or couldn't be read
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
Started analysis of SRR1810155.fastq.gz
application/gzip
Started analysis of SRR1810163.fastq.gz
Started analysis of SRR1810165.fastq.gz
Started analysis of SRR1810162.fastq.gz
Started analysis of SRR1810166.fastq.gz
Started analysis of SRR1810161.fastq.gz
Started analysis of SRR1810164.fastq.gz
Started analysis of SRR1810160.fastq.gz
Started analysis of SRR1810156.fastq.gz
Started analysis of SRR1

FastQC will output the results in HTML format, as below, for all forward and reverse reads.

In [11]:
from IPython.display import IFrame
IFrame(src='data/fastqc/SRR1810154_fastqc.html', width=800, height=600)

Although it is best practice to look over the quality reports individually, tools like MultiQC allow one to quickly look at a combined summary of the quality reports of all fastq files.

For instance, the below table shows all warnings, passes, or failures, from each FastQC report. There are other summaries created as well by MultiQC.

In [12]:
!multiqc -f data/fastqc/

import pandas as pd
dframe = pd.read_csv("./multiqc_data/multiqc_fastqc.txt", sep='\t')
display(dframe)


/// ]8;id=195231;https://multiqc.info\MultiQC]8;;\ 🔍 v1.33

       file_search | Search path: /home/jupyter/Bulk-and-Single-Cell-RNAseq/Bulk_RNA-Seq_Tutorials/Bulk_RNA-Seq_Mouse/data/fastqc
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 34/34  data/fastqc/SRR1810164_fastqc.html
            fastqc | Found 17 reports
     write_results | Data        : multiqc_data
     write_results | Report      : multiqc_report.html
           multiqc | MultiQC complete


,Sample,Filename,File type,Encoding,Total Sequences,Total Bases,Sequences flagged as poor quality,Sequence length,%GC,total_deduplicated_percentage,...,basic_statistics,per_base_sequence_quality,per_sequence_quality_scores,per_base_sequence_content,per_sequence_gc_content,per_base_n_content,sequence_length_distribution,sequence_duplication_levels,overrepresented_sequences,adapter_content
0,SRR1810154,SRR1810154.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,25175967.0,1.7 Gbp,0.0,25-75,47.0,86.955798,...,pass,fail,pass,warn,pass,fail,warn,pass,pass,pass
1,SRR1810155,SRR1810155.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,17909879.0,1.1 Gbp,0.0,25-75,49.0,90.986817,...,pass,fail,pass,fail,pass,fail,warn,pass,pass,pass
2,SRR1810156,SRR1810156.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,17088302.0,1.1 Gbp,0.0,25-75,49.0,90.536439,...,pass,fail,pass,fail,pass,fail,warn,pass,pass,pass
3,SRR1810157,SRR1810157.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,30918887.0,2.1 Gbp,0.0,25-75,47.0,88.664267,...,pass,fail,pass,fail,pass,fail,warn,pass,pass,pass
4,SRR1810158,SRR1810158.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,23414444.0,1.5 Gbp,0.0,25-75,48.0,88.821027,...,pass,fail,pass,fail,pass,fail,warn,pass,pass,pass
5,SRR1810159,SRR1810159.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,23475881.0,1.5 Gbp,0.0,25-75,47.0,89.128310,...,pass,fail,pass,fail,pass,fail,warn,pass,pass,pass
6,SRR1810160,SRR1810160.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,21117576.0,1.4 Gbp,0.0,25-75,45.0,88.674398,...,pass,fail,pass,fail,pass,fail,warn,pass,pass,pass
7,SRR1810161,SRR1810161.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,26829920.0,1.8 Gbp,0.0,25-75,46.0,88.678457,...,pass,fail,pass,fail,pass,fail,warn,pass,pass,pass
8,SRR1810162,SRR1810162.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,22942035.0,1.5 Gbp,0.0,25-75,48.0,89.975982,...,pass,fail,pass,fail,pass,fail,warn,pass,pass,pass
9,SRR1810163,SRR1810163.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,23907580.0,1.5 Gbp,0.0,25-75,50.0,91.299851,...,pass,fail,pass,fail,pass,fail,warn,pass,pass,pass


### STEP 5: Run Trimmomatic

Trimmomatic will trim off any adapter sequences or low quality sequence it detects in the FASTQ files.

Using piping and our original list, it is possible to queue up a batch run of Trimmomatic for all our files, note that this is a different way to run a loop compared with what we did before.

The below code may take approximately 30 minutes to run.

In [15]:
!cat accs.txt | xargs -I {} \
trimmomatic SE -threads $THREADS \
'data/raw_fastq/{}.fastq.gz' \
'data/trimmed/{}_trimmed.fastq' \
ILLUMINACLIP:data/trimmed/TruSeq3-SE.fa:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36

TrimmomaticSE: Started with arguments:
 -threads 31 data/raw_fastq/SRR1810170.fastq.gz data/trimmed/SRR1810170_trimmed.fastq ILLUMINACLIP:data/trimmed/TruSeq3-SE.fa:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36
ILLUMINACLIP: Using adapter file from current working directory: /home/jupyter/Bulk-and-Single-Cell-RNAseq/Bulk_RNA-Seq_Tutorials/Bulk_RNA-Seq_Mouse/data/trimmed/TruSeq3-SE.fa
Using Long Clipping Sequence: 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTA'
Using Long Clipping Sequence: 'AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC'
ILLUMINACLIP: Using 0 prefix pairs, 2 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Reads: 28796083 Surviving: 17179055 (59.66%) Dropped: 11617028 (40.34%)
TrimmomaticSE: Completed successfully
TrimmomaticSE: Started with arguments:
 -threads 31 data/raw_fastq/SRR1810171.fastq.gz data/trimmed/SRR1810171_trimmed.fastq ILLUMINACLIP:data/trimmed/TruSeq3-SE.fa:2:30:10 LEADING:3 TRAILING:3 SLID

## STEP 5: Run FastQC
It's best practice to run FastQC after trimming. However, you may decide to run FastQC only once, before or after trimming.

We will proceed with only the forward reads -- this is because, looking at Trimmomatic, there were very few 'orphaned' reads. That is to say, most forward and reverse reads were successfully paired together. Because we are just trying to map to a reference genome, the read lengths of the forward reads alone, in this case, around 60 million basepairs, should be sufficient.

The below code may take around 15-20 minutes to run.

In [16]:
# Run FastQC
!cat accs.txt | xargs -P $THREADS -I {} fastqc data/trimmed/{}_trimmed.fastq -o data/fastqc_trimmed/

Skipping 'data/trimmed/SRR1810174_trimmed.fastq' which didn't exist, or couldn't be read
Skipping 'data/trimmed/SRR1810171_trimmed.fastq' which didn't exist, or couldn't be read
Skipping 'data/trimmed/SRR1810172_trimmed.fastq' which didn't exist, or couldn't be read
Skipping 'data/trimmed/SRR1810173_trimmed.fastq' which didn't exist, or couldn't be read
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
Started analysis of SRR1810155_trimmed.fastq
null
Started analysis of SRR1810157_trimmed.fastq
Started analysis of SRR1810167_trimmed.fastq
Started analysis of SRR1810154_trimmed.fastq
Started analysis of SRR1810158_trimmed.fastq
null
Started analysis of SRR1810170_trimmed.fastq
Started analysis of SRR1810164_trimmed.fastq
Started analysis of SRR1810162_trimmed.fastq
Started analysis of SRR1810168_trimmed.fastq
Started analysis of SRR1810165_trimmed.fastq
Started analysis of SRR1810160_trimmed.fastq
Started analysis of SRR1810169_trimmed.fastq
Started analysis of

## STEP 6: Run MultiQC
MultiQC reads in the FastQC reports and generates a compiled report for all the analyzed FASTQ files.

In [17]:
#!multiqc -f data/fastqc_samples/
!multiqc -f -o data/multiqc_samples/ data/fastqc_trimmed/


/// ]8;id=917870;https://multiqc.info\MultiQC]8;;\ 🔍 v1.33

       file_search | Search path: /home/jupyter/Bulk-and-Single-Cell-RNAseq/Bulk_RNA-Seq_Tutorials/Bulk_RNA-Seq_Mouse/data/fastqc_trimmed
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 34/34  lm  
            fastqc | Found 17 reports
     write_results | Data        : data/multiqc_samples/multiqc_data
     write_results | Report      : data/multiqc_samples/multiqc_report.html
           multiqc | MultiQC complete


## STEP 7: Preparing the STAR-Compatible RSEM Reference

This command prepares a reference genome and annotation files for RNA-Seq analysis using RSEM (RNA-Seq by Expectation-Maximization) and STAR (Spliced Transcripts Alignment to a Reference). It generates files needed to quantify gene and isoform expression. The rsem-prepare-reference function takes a GTF file with gene annotations (mouse_annotation.gtf) and a FASTA file with the reference genome sequence (mouse_genome.fa). It processes these files to create a reference, saving the output in the mouse_reference directory. The --star option ensures the reference is compatible with STAR for efficient transcriptome alignment. The -p $THREADS option sets the number of threads used for parallel processing, speeding up the preparation process.

In [18]:
# Preparing the reference genome
!rsem-prepare-reference --gtf data/reference/mouse_annotation.gtf --star -p $THREADS data/reference/mouse_genome.fa data/rsem_reference/mouse_reference > /dev/null

## STEP 8: Run STAR for Alignment, Prepare and Run RSEM for Quantification

This script automates RNA-Seq gene expression quantification using RSEM and STAR. It reads SRR accession IDs from accs.txt, saves results in data/rsem_output, and runs rsem-calculate-expression for each ID. It uses paired-end trimmed FASTQ files from data/trimmed/ and a STAR-aligned RSEM reference (mouse_reference).

In [ ]:
import os

# Ensure you set the path to the RSEM binary
# Read the SRR accessions from the file
with open('accs.txt', 'r') as f:
    srr_accessions = [line.strip() for line in f.readlines()]

# Define the output directory
output_dir = "data/rsem_output"

# Loop through each SRR accession and run rsem-calculate-expression
for srr in srr_accessions:
    !rsem-calculate-expression -p $THREADS --star \
    data/trimmed/{srr}_trimmed.fastq data/rsem_reference/mouse_reference data/rsem_output/{srr} > /dev/null


EXITING because of fatal input ERROR: could not open readFilesIn=data/trimmed/SRR1810171_trimmed.fastq

Jan 07 16:52:16 ...... FATAL ERROR, exiting

EXITING because of fatal input ERROR: could not open readFilesIn=data/trimmed/SRR1810172_trimmed.fastq

Jan 07 16:52:17 ...... FATAL ERROR, exiting

EXITING because of fatal input ERROR: could not open readFilesIn=data/trimmed/SRR1810173_trimmed.fastq

Jan 07 16:52:17 ...... FATAL ERROR, exiting

EXITING because of fatal input ERROR: could not open readFilesIn=data/trimmed/SRR1810174_trimmed.fastq

Jan 07 16:52:17 ...... FATAL ERROR, exiting


### STEP 11: Report the expression of ENSMUSG00000064351 for each file

Use `grep` to report the expression in the wild-type sample. The fields in the RSEM `genes.results` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
- `Name`
- `Length`
- `EffectiveLength`
- `TPM`
- `NumReads`

In [ ]:
!grep 'ENSMUSG00000064351' data/rsem_output/*.genes.results

## Conclusion

In this tutorial, we covered the following key concepts and workflow steps:

- **Bulk RNA-seq Preprocessing**: Downloading the full dataset and metadata using the SRA Toolkit and Athena, and setting up directories
- **Quality Control**: Use FastQC and MultiQC to assess the quality of reads in the dataset and combine results for all samples to generate a comprehensive overview of quality metrics across multiple samples.
- **Adapter Trimming**: Learn how to use Trimmomatic to remove adapter sequences and low-quality bases from FASTQ reads.
- **Read Mapping and Quantification**: Understand the purpose of indexing and learn how to use STAR to create an index of the reference genome for efficient read mapping. Map reads to reference genome and quantify gene expression levels using RSEM.

In summary, this Jupyter Notebook provided a hands-on demonstration of an extended Bulk RNA-Seq analysis workflow, guiding users through essential steps such as obtaining sequencing data from the Sequence Read Archive using the SRA Toolkit and Athena, read trimming with Trimmomatic, quality control with FastQC and MultiQC, reference genome indexing with STAR, read mapping and quantification using RSEM, and storing data on an AWS S3 bucket. By processing the full set of reads from a mouse dataset, we were able to observe the expression levels of the top 10 most highly expressed genes in each sample. This workflow serves as a foundation for more advanced analyses, and further resources are available for utilizing R/DESeq2 for differential gene expression analysis using the read counts generated in this tutorial, and using NetAct for transcription factor network analysis. Ultimately, this tutorial equips users with the basic skills to analyze RNA-seq data and to understand the core components of a typical RNA-seq pipeline.



## <a name="workflow">Additional Workflows</a>

Now that you have read counts per gene, feel free to explore the R workflow which creates plots and analyses using these readcount files, or try other alternate workflows for creating read count files, such as using snakemake.


[Workflow One:](Tutorial_1_subsampling_mouse-miniforge.ipynb) A short introduction to downloading and mapping sequences to a mouse genome using STAR and RSEM.


[Workflow Two (DEG Analysis):](Tutorial_2_DEG_Analysis_mouse.ipynb) Using Deseq2 and R to conduct clustering and differential gene expression analysis.

[Workflow Three (Network Analysis):](Tutorial_3_NetAct.ipynb) Using NetAct and R to conduct transcription factor network analysis.
